In [134]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import os

athlete_no = '6483989' # Anna Foster
#athlete_no = '3013148' # Carolyn Birkenfeld
athlete_no = '5824213' # Lexi Kyro
url = 'https://www.athletic.net/TrackAndField/Athlete.aspx?AID='+athlete_no+'#!/L0'

response = requests.get(url)

In [135]:
chromedriver = "/usr/local/bin/chromedriver"

os.environ["webdriver.chrome.driver"] = chromedriver

driver = webdriver.Chrome(chromedriver)
driver.get("https://www.athletic.net")
driver.get("https://www.athletic.net/account/login/?ReturnUrl=%2Fdefault.aspx")

#%load_ext dotenv
#%dotenv
EMAIL = 'adds@silverbeach.com'
PASSWORD = os.environ.get('PASSWORD')
username_form = driver.find_element_by_name("email")
username_form.send_keys(EMAIL) # enter email

username_form.send_keys(Keys.RETURN)
password_form=driver.find_element_by_name("password") # note another approach
password_form.send_keys(PASSWORD) # enter password
password_form.send_keys(Keys.RETURN)

## Using Pandas

In [136]:
header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

r = requests.get(url, headers=header)

dfs = pd.read_html(r.text)

In [137]:
for df in dfs:
    if df[0][0] == '400 Meters':
        all_years = df
        
print(all_years)

              0     1             2
0    400 Meters   NaN           NaN
1  2014 Outdoor   9.0       1:02.20
2  2015 Outdoor  10.0       1:06.13
3  2016 Outdoor  11.0  1:02.04 PR *
4  2017 Outdoor  12.0       1:02.59


In [138]:
dfs[2]

,0,1,2
0,400 Meters - Relay Split,NaN,NaN
1,2014 Outdoor,9.0,1:03.94c
2,2015 Outdoor,10.0,1:07.64c
3,2016 Outdoor,11.0,1:02.54c PR *
4,2017 Outdoor,12.0,1:06.04c


### Pull data for several athletes

Loop over a list of athletes from a csv file and read their 400 Meters numbers

In [ ]:
header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

In [142]:
file = '/Users/dana/metis/git/Metis_Bootcamp/Luther_Project/athletes.csv'
athlete_list = pd.read_csv(file)

In [199]:
all_data = athlete_list[['ID','Name','School']]

for ID in athlete_list['ID']:
#    print(all_data[all_data['ID'==ID]])
    ID_str = str(ID)
    url = 'https://www.athletic.net/TrackAndField/Athlete.aspx?AID='+ID_str+'#!/L0'
    r = requests.get(url, headers=header)
    dfs = pd.read_html(r.text)
    for df in dfs:
        if df[0][0] == '400 Meters':
            df.drop(df.index[0:1],inplace=True)
            print(df)
            for index, row in df.iterrows():
                if 'Indoor' not in str(row[0]):
                    print(f"grade {int(row[1])}, time {row[2]}")
                    grade = int(row[1])
                    time = cleantime(str(row[2]))
                    print(grade,time)
#                print(row[1],row[2],row[3],row[4])
#            (athlete_list[athlete_list['Name'] == 'Caroline Dolan']
#             ['times']=df)
            


              0     1           2
1  2015 Outdoor   9.0     1:04.08
2  2016 Outdoor  10.0   1:00.22 *
3  2017 Outdoor  11.0  58.21 PR *
4   2018 Indoor  12.0  1:01.82 PR
5  2018 Outdoor  12.0       59.50
grade 9, time 1:04.08
9 64.08
grade 10, time 1:00.22 *
10 60.22
grade 11, time 58.21 PR *
11 58.21
grade 12, time 59.50
12 59.5
              0     1           2
1  2018 Outdoor  10.0  1:00.60 PR
grade 10, time 1:00.60 PR
10 60.6
              0     1           2
1  2015 Outdoor   9.0     1:02.63
2  2016 Outdoor  10.0     59.37 *
3  2017 Outdoor  11.0     1:01.21
4   2018 Indoor  12.0  1:02.28 PR
5  2018 Outdoor  12.0  58.95 PR *
grade 9, time 1:02.63
9 62.63
grade 10, time 59.37 *
10 59.37
grade 11, time 1:01.21
11 61.21
grade 12, time 58.95 PR *
12 58.95
              0     1           2
1  2017 Outdoor   9.0  1:01.18 PR
2  2018 Outdoor  10.0     1:01.25
grade 9, time 1:01.18 PR
9 61.18
grade 10, time 1:01.25
10 61.25
              0     1           2
1  2017 Outdoor   9.0  1:00.65 P

In [196]:
def cleantime(time_str):
    time_str = time_str.replace('PR','')
    time_str = time_str.replace('*','')
    time_str = time_str.strip()
    if ':' in time_str:
        t = time_str.split(':')
        time = float(t[0])*60 + float(t[1])
    else: 
        time = float(time_str)
    return(time)
    
time_str = '47.31 PR'
cleantime(time_str)

47.31

In [161]:
athlete_list

,ID,Name,Time,School,Grade,test
0,7441281,Caroline Dolan,59.60a,Stanwood,12,0
1,13736186,Eloika Schemmel,1:00.91a,Shorecrest,10,0
2,7880535,Ronna Iverson,1:01.41a,Shorecrest,12,0
3,11097622,Skye Nephew,1:01.56a,Shorewood,10,0
4,11053203,Nicole Headrick,1:02.36a,Ferndale,10,0
5,7853471,Adrienne Cordell,1:02.38a,Shorewood,12,0
6,11166388,Sesley Love,1:02.98a,Meadowdale,10,0
7,12870444,Courtney VanGiesen,1:03.64a,Oak Harbor,9,0
8,9213539,Allyson McArthur,1:03.64a,Lynnwood,11,0
9,9408152,Jessica Vester,1:05.33a,Oak Harbor,11,0
